In [1]:
from sklearn.decomposition import FactorAnalysis
from scipy.stats import pearsonr
from bids import BIDSLayout
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from util.tasks import SoPA_loadings, SoNA_loadings
loadings = np.stack([SoPA_loadings, SoNA_loadings])

In [2]:
layout = BIDSLayout('data_bids_anon', validate = False)

In [3]:
def load_sub(sub):
    f = layout.get(subject = sub, task = 'SoAScale')[0]
    df = f.get_df()
    # exclude if only replied 1 or 7
    if df.response.isin([1, 7]).all():
        exclude = True
    elif (df.response[0] == df.response).all(): # or always said same thing
        exclude = True
    else:
        exclude = False
    return df.response.to_numpy(), exclude

subs = layout.get_subjects(task = 'SoAScale')
data = [load_sub(s) for s in subs]
X, exclude = list(zip(*data))
X, exclude = np.stack(X), np.array(exclude)
X = X[~exclude]

In [4]:
def score_loadings(X, loadings, zero_out = None):
    '''
    computes the correlation between features and their 
    expected values given the factor loadings 
    '''
    X = X.copy()
    factors = X @ loadings.T # project observations onto factors
    if not zero_out is None:
        factors[:, zero_out] = factors[:, zero_out].mean()
    X_recon = factors @ loadings # project factors back to data
    corr = [pearsonr(X[:,j], X_recon[:,j]).statistic for j in range(X.shape[1])]
    return np.array(corr)

score_loadings(X, loadings)

array([0.7388275 , 0.76243121, 0.78281459, 0.613707  , 0.67906274,
       0.74147223, 0.74148276, 0.6902449 , 0.66263242, 0.68360632,
       0.72146164, 0.57357142, 0.61639507])

In [5]:
def shuffle_features(X, seed = None):
    '''
    Shuffles each column of X individually to generate a draw 
    from the same randomization null used in "parallel analysis" [1] 
    evaluating factor structures. This idea is that by shuffling
    each of the features individually, you destroy the factor 
    structure while preserving the mean and noise properties of
    the features.

    References
    -------------
    [1] Dobriban, E. (2020). Permutation methods for factor analysis and PCA.
    '''
    rng = np.random.default_rng(seed)
    X = X.copy()
    for i in range(X.shape[1]):
        rng.shuffle(X[:,i])
    return X

shuffle_features(X).shape

(174, 13)

In [6]:
def test_factor(exclude_factor):

    ## permutation-based confirmatory factor analysis 
    # score a priori (out-of-sample) loadings
    score = score_loadings(X, loadings, exclude_factor)
    H0 = [score]
    for i in range(1000):
        # then compute loadings estimated in-sample but on shuffled data
        fa = FactorAnalysis(n_components = 2, rotation = 'quartimax', random_state = i)
        Y = shuffle_features(X, seed = i)
        fa = fa.fit(Y) # fit new loadings to shuffled features
        H0.append(score_loadings(X, fa.components_, exclude_factor)) # and score on unshuffled
    H0 = np.stack(H0)
    p = (H0 > score).mean(0)

    print('Items related to factor structure:\n')
    res = layout.get(task = 'SoAScale')[0].get_df().question[p <= .05]
    for item, quest, r, _p in zip(res.index + 1, res, score[p <= .05], p[p <= .05]):
        print('Item %d (r = %.03f, p = %.03f): %s'%(item, r, _p, quest))
    print('\nItems possibly unrelated to factor structure:\n')
    res = layout.get(task = 'SoAScale')[0].get_df().question[p > .05]
    for item, quest, r, _p in zip(res.index + 1, res, score[p > .05], p[p > .05]):
        print('Item %d (r = %.03f, p = %.03f): %s'%(item, r, _p, quest))
    p_all = (H0.mean(1) >= score.mean()).mean()
    print('\np = %.05f for whole factor model'%p_all)

## sense of negative agency
test_factor(0)

Items related to factor structure:

Item 3 (r = 0.783, p = 0.008): My actions just happen without my intention
Item 7 (r = 0.740, p = 0.050): The outcomes of my actions generally surprise me
Item 10 (r = 0.694, p = 0.024): Nothing I do is actually voluntary
Item 11 (r = 0.745, p = 0.045): While I am in action, I feel like I am a remote controlled robot

Items possibly unrelated to factor structure:

Item 1 (r = 0.523, p = 0.116): I am in full control of what I do
Item 2 (r = 0.743, p = 0.059): I am just an instrument in the hands of somebody or something else
Item 4 (r = 0.460, p = 0.113): I am the author of my actions
Item 5 (r = 0.687, p = 0.185): The consequences of my actions feel like they don't logically follow my actions
Item 6 (r = 0.719, p = 0.389): My movements are automatic-- my body simply makes them
Item 8 (r = -0.277, p = 0.975): Things I do are subjects only to my free will
Item 9 (r = 0.455, p = 0.119): The decision whether and when to act is within my hands
Item 12 (r 

In [7]:
## sense of positive agency
test_factor(1)

Items related to factor structure:

Item 1 (r = 0.727, p = 0.000): I am in full control of what I do
Item 2 (r = 0.550, p = 0.005): I am just an instrument in the hands of somebody or something else
Item 4 (r = 0.624, p = 0.000): I am the author of my actions
Item 5 (r = 0.453, p = 0.007): The consequences of my actions feel like they don't logically follow my actions
Item 8 (r = 0.681, p = 0.003): Things I do are subjects only to my free will
Item 9 (r = 0.665, p = 0.001): The decision whether and when to act is within my hands
Item 10 (r = 0.320, p = 0.008): Nothing I do is actually voluntary
Item 11 (r = 0.297, p = 0.011): While I am in action, I feel like I am a remote controlled robot
Item 13 (r = 0.625, p = 0.000): I am completely responsible for everything that results from my actions

Items possibly unrelated to factor structure:

Item 3 (r = -0.452, p = 0.999): My actions just happen without my intention
Item 6 (r = -0.232, p = 0.998): My movements are automatic-- my body simp

In [8]:
from shap import LinearExplainer
from shap.maskers import Partition
np.random.seed(0)

## estimate Shapley values to quantify impact of features on factor values
# (I like this approach but I'm not sure anyone else using psychometric 
# scales knows what it is so)

explainer = LinearExplainer((loadings[0], 0), masker = Partition(X))
shap = explainer.shap_values(X)
shap = np.abs(shap).mean(0)
print('Positive agency:')
print('------------------')
for item, s in zip(np.arange(1, shap.size + 1), shap):
    print('Item %d: shapley = %.03f'%(item, s))

explainer = LinearExplainer((loadings[1], 0), masker = Partition(X))
shap = explainer.shap_values(X)
shap = np.abs(shap).mean(0)
print('\nNegative agency:')
print('------------------')
for item, s in zip(np.arange(1, shap.size + 1), shap):
    print('Item %d: shapley = %.03f'%(item, s))

Positive agency:
------------------
Item 1: shapley = 0.566
Item 2: shapley = 0.241
Item 3: shapley = 0.008
Item 4: shapley = 0.329
Item 5: shapley = 0.318
Item 6: shapley = 0.218
Item 7: shapley = 0.011
Item 8: shapley = 0.847
Item 9: shapley = 0.408
Item 10: shapley = 0.078
Item 11: shapley = 0.115
Item 12: shapley = 1.021
Item 13: shapley = 0.512

Negative agency:
------------------
Item 1: shapley = 0.060
Item 2: shapley = 0.483
Item 3: shapley = 0.543
Item 4: shapley = 0.291
Item 5: shapley = 0.465
Item 6: shapley = 0.886
Item 7: shapley = 0.611
Item 8: shapley = 0.127
Item 9: shapley = 0.193
Item 10: shapley = 0.492
Item 11: shapley = 0.544
Item 12: shapley = 0.032
Item 13: shapley = 0.040
